<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Otros/22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy

     - 2.7 MB 9.3 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (1

In [32]:
%%writefile app.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d
import requests
from io import BytesIO
import yt_dlp as youtube_dl
import subprocess
import os
import shutil

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 3: Modulación en Amplitud (AM) con Audio"))
if punto == "Punto 3: Modulación en Amplitud (AM) con Audio":
    st.write("### Punto 3: Modulación en Amplitud (AM) con Audio")

    # Función para descargar audio de YouTube en formato MP3
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    return filename

# Función para convertir MP3 a WAV usando ffmpeg
def convert_mp3_to_wav(mp3_filename, wav_filename):
    subprocess.call(['ffmpeg', '-y', '-i', mp3_filename, wav_filename])

# Función para eliminar los archivos temporales de la canción anterior
def clear_previous_files(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

# Interfaz de Streamlit
st.title("Modulación de Audio desde YouTube y Escuchar Segmento")

# Directorio temporal para guardar archivos de audio
temp_dir = "temp_audio"
clear_previous_files(temp_dir)  # Asegurarse de que no haya archivos previos

# Entrada para el enlace de YouTube
video_url = st.text_input("Ingresa el enlace de una canción en YouTube:")

# Parámetros ajustables con sliders
start_time = st.slider("Tiempo de inicio del segmento (segundos)", 0, 60, 35)     # Tiempo de inicio del segmento
#duration = st.slider("Duración del segmento (segundos)", 1, 10, 5)                # Duración del segmento

# Procesamiento del archivo de audio si se ha ingresado un enlace
if video_url:
    try:
        # Limpiar archivos anteriores
        clear_previous_files(temp_dir)

        # Descargar y convertir el archivo de YouTube
        name = "audio_downloaded"
        mp3_filename = os.path.join(temp_dir, f"{name}.mp3")
        wav_filename = os.path.join(temp_dir, f"{name}.wav")

        st.write("Descargando y procesando el audio...")
        download_ytvid_as_mp3(video_url, mp3_filename)
        convert_mp3_to_wav(mp3_filename, wav_filename)

        # Leer el archivo WAV
        x, fm = sf.read(wav_filename)
        x_mono = np.mean(x, axis=1) if x.ndim > 1 else x
        ns = len(x_mono) / fm

        # Segmento seleccionado por el usuario
        end_time = start_time + 5
        start_index = int(start_time * fm)
        end_index = int(end_time * fm)

        x_mono_segment = x_mono[start_index:end_index]
        t_segment = np.arange(start_time, end_time, 1 / fm)

        # Reproducir el segmento seleccionado
        st.write(f"Reproduciendo segmento de {start_time} a {end_time} segundos.")

        # Guardar el segmento seleccionado como archivo temporal WAV
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=temp_dir) as temp_wav:
            sf.write(temp_wav.name, x_mono_segment, fm)
            st.audio(temp_wav.name, format='audio/wav')

        # Generar la señal portadora
        Ac = max(abs(x_mono_segment)) / 0.8  # Amplitud de la portadora
        c_segment = Ac * np.sin(2 * np.pi * 10000 * t_segment)  # Señal portadora

        # Señal modulada AM
        y = (1 + x_mono_segment / Ac) * c_segment

        # Graficar señal portadora y mensaje
        st.write("### Señal de Portadora y Mensaje")
        fig, ax = plt.subplots()
        ax.plot(t_segment, c_segment, label='Portadora', alpha=0.7)
        ax.plot(t_segment, x_mono_segment, label='Mensaje', alpha=0.7)
        ax.set_xlabel('Tiempo (s)')
        ax.set_ylabel('Amplitud')
        ax.legend()
        st.pyplot(fig)

        # Graficar la señal modulada
        st.write("### Señal Modulada en AM")
        fig, ax = plt.subplots()
        ax.plot(t_segment, y, label='Modulada')
        ax.set_xlabel('Tiempo (s)')
        ax.set_ylabel('Amplitud')
        st.pyplot(fig)

        # Espectros de las señales
        st.write("### Espectros de las Señales")
        Xfc = np.fft.rfft(c_segment)
        Xfm = np.fft.rfft(x_mono_segment)
        Xfy = np.fft.rfft(y)
        vfre = np.fft.rfftfreq(len(c_segment), 1 / fm)

        fig, ax = plt.subplots()
        ax.plot(vfre, np.abs(Xfy), label='Modulada', alpha=0.7)
        ax.plot(vfre, np.abs(Xfm), label='Mensaje', alpha=0.7)
        ax.plot(vfre, np.abs(Xfc), label='Portadora', alpha=0.7)
        ax.set_xlabel('Frecuencia [Hz]')
        ax.set_ylabel('|X(f)|')
        ax.set_ylim([min(np.abs(Xfm)), 1.1 * max(np.abs(Xfm))])
        ax.legend()
        ax.grid(True)
        st.pyplot(fig)

    except Exception as e:
        st.error(f"Error al procesar el audio: {e}")




Overwriting app.py


In [33]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://f792-35-231-185-114.ngrok-free.app" -> "http://localhost:5011"
